In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

### Cheatsheet
- Residual: JKG
- df: derajat kebebasan
- mean_sq = kuadrat_rata"
- sum_sq:  jumlah kuadrat
- PR(>F): p-value (tingkat signifikansi)

- p-value < alpha --> Ada pengaruh signifikan
- p-value > alpha --> Tidak ada pengaruh signifikan

In [2]:
# Data diambil dari tabel
data = {
    'C2F6_Flow_Rate': [1]*6 + [2]*6 + [3]*6,
    'Power_Supplied': [1,1,2,2,3,3]*3,
    'Kecepatan_Etsa': [
        288, 360, 488, 465, 670, 720,   # Flow rate 1
        385, 411, 482, 521, 692, 724,   # Flow rate 2
        488, 462, 595, 612, 761, 801    # Flow rate 3
    ]
}

df = pd.DataFrame(data)

# Lakukan ANOVA dua arah
model = ols('Kecepatan_Etsa ~ C(C2F6_Flow_Rate) + C(Power_Supplied) + C(C2F6_Flow_Rate):C(Power_Supplied)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
C(C2F6_Flow_Rate),46343.111111,2.0,29.794128,1.073909e-04
C(Power_Supplied),330003.444444,2.0,212.160226,2.681946e-08
C(C2F6_Flow_Rate):C(Power_Supplied),3162.222222,4.0,1.016501,4.484601e-01
Residual,6999.500000,9.0,NaN,NaN


In [5]:
import numpy as np
import pandas as pd
from scipy import stats

# Data dari gambar
data = {
    'Obat': ['A']*8 + ['B']*8 + ['C']*8,
    'Waktu': [
        5, 4, 8, 6, 3, 3, 5, 2,   # Obat A
        9, 7, 8, 6, 9, 3, 7, 4,   # Obat B
        7, 6, 9, 4, 7, 2, 3, 4    # Obat C
    ]
}
df = pd.DataFrame(data)

# Pisahkan data tiap kelompok
A = df[df['Obat'] == 'A']['Waktu']
B = df[df['Obat'] == 'B']['Waktu']
C = df[df['Obat'] == 'C']['Waktu']

# ANOVA dengan scipy
f_stat, p_val = stats.f_oneway(A, B, C)

# Derajat bebas
k = 3                      # Jumlah kelompok
n = 24                     # Total data
df_total = n - 1
df_between = k - 1
df_within = n - k

# Jumlah kuadrat total
grand_mean = df['Waktu'].mean()
ss_total = ((df['Waktu'] - grand_mean)**2).sum()

# Jumlah kuadrat antar kelompok (between)
mean_A = A.mean()
mean_B = B.mean()
mean_C = C.mean()
ss_between = 8*((mean_A - grand_mean)**2 + (mean_B - grand_mean)**2 + (mean_C - grand_mean)**2)

# Jumlah kuadrat dalam kelompok (within/error)
ss_within = ((A - mean_A)**2).sum() + ((B - mean_B)**2).sum() + ((C - mean_C)**2).sum()

# Mean square
ms_between = ss_between / df_between
ms_within = ss_within / df_within

# F Hitung manual
F_manual = ms_between / ms_within

# Print solusi
print("Derajat bebas total:", df_total)
print("Derajat bebas galat/error:", df_within)
print("Jumlah kuadrat total:", round(ss_total, 2))
print("Jumlah kuadrat antara kelompok:", round(ss_between, 2))
print("Jumlah kuadrat dalam/error:", round(ss_within, 2))
print("Nilai F Hitung (manual):", round(F_manual, 2))
print("Nilai F Hitung (scipy):", round(f_stat, 2))
print("p-value:", p_val)

Derajat bebas total: 23
Derajat bebas galat/error: 21
Jumlah kuadrat total: 117.96
Jumlah kuadrat antara kelompok: 18.58
Jumlah kuadrat dalam/error: 99.38
Nilai F Hitung (manual): 1.96
Nilai F Hitung (scipy): 1.96
p-value: 0.16529487831932194


In [6]:
import pandas as pd
from scipy import stats

# Data dari gambar
data = {
    "Campuran": ["A"]*12 + ["B"]*12 + ["C"]*12 + ["D"]*12,
    "SerapAir": [
        # Campuran A
        551, 457, 450, 731, 499, 632, 552, 458, 451, 732, 500, 633,
        # Campuran B
        595, 580, 508, 583, 633, 517, 596, 581, 509, 584, 634, 518,
        # Campuran C
        639, 615, 511, 573, 648, 677, 640, 616, 512, 574, 649, 678,
        # Campuran D
        417, 449, 517, 438, 415, 555, 418, 450, 518, 439, 416, 566
    ]
}
df = pd.DataFrame(data)

# Pisahkan data tiap kelompok
A = df[df['Campuran'] == 'A']['SerapAir']
B = df[df['Campuran'] == 'B']['SerapAir']
C = df[df['Campuran'] == 'C']['SerapAir']
D = df[df['Campuran'] == 'D']['SerapAir']

# ANOVA satu arah
F_stat, p_value = stats.f_oneway(A, B, C, D)

# Jumlah kuadrat total (JKG)
grand_mean = df['SerapAir'].mean()
JKG = ((df['SerapAir'] - grand_mean)**2).sum()

# Print hasil
print(f"F-hitung: {F_stat:.2f}")
print(f"p-value: {p_value:.5f}")
print(f"JKG (Jumlah Kuadrat Total): {JKG:.2f}")

if p_value < 0.05:
    print("Tolak H0: Ada pengaruh jenis campuran beton terhadap serapan air.")
else:
    print("Terima H0: Tidak ada pengaruh jenis campuran beton terhadap serapan air.")

F-hitung: 9.07
p-value: 0.00009
JKG (Jumlah Kuadrat Total): 348607.92
Tolak H0: Ada pengaruh jenis campuran beton terhadap serapan air.


In [ ]:
import pandas as pd
import scipy.stats as stats

# Data
data = {
    "Campuran": ["A"]*12 + ["B"]*12 + ["C"]*12 + ["D"]*12,
    "SerapAir": [
        # Campuran A
        551, 457, 450, 731, 499, 632, 556, 462, 455, 736, 504, 637,
        # Campuran B
        595, 580, 508, 583, 633, 517, 600, 585, 513, 588, 638, 522,
        # Campuran C
        639, 615, 511, 573, 648, 677, 644, 620, 516, 578, 653, 682,
        # Campuran D
        417, 449, 517, 438, 415, 555, 422, 454, 522, 443, 420, 560
    ]
}

# Buat DataFrame
df = pd.DataFrame(data)

# Lihat 5 baris awal
print(df.head())

# Pisahkan data berdasarkan kelompok Campuran
group_A = df[df['Campuran'] == 'A']['SerapAir']
group_B = df[df['Campuran'] == 'B']['SerapAir']
group_C = df[df['Campuran'] == 'C']['SerapAir']
group_D = df[df['Campuran'] == 'D']['SerapAir']

# Lakukan ANOVA satu arah
F_statistic, p_value = stats.f_oneway(group_A, group_B, group_C, group_D)

print(f"F-hitung = {F_statistic}")
print(f"p-value = {p_value}")

# Hitung JKG (Jumlah Kuadrat Grup)
grand_mean = df['SerapAir'].mean()

# Mean tiap grup
mean_A = group_A.mean()
mean_B = group_B.mean()
mean_C = group_C.mean()
mean_D = group_D.mean()

# Banyak data tiap grup
n = len(group_A)  # sama semua karena 12 data per grup

# JKG formula: sum n * (mean_group - grand_mean)^2
JKG = n * ((mean_A - grand_mean)**2 + (mean_B - grand_mean)**2 + (mean_C - grand_mean)**2 + (mean_D - grand_mean)**2)
print(f"JKG = {JKG}")

# Kesimpulan berdasarkan p-value
alpha = 0.05
if p_value < alpha:
    print("Tolak H0: Ada pengaruh jenis campuran beton terhadap serap air.")
else:
    print("Terima H0: Tidak ada pengaruh jenis campuran beton terhadap serap air.")


  Campuran  SerapAir
0        A       551
1        A       457
2        A       450
3        A       731
4        A       499
F-hitung = 9.266786782285758
p-value = 7.250170845347337e-05
JKG = 134893.66666666663
Tolak H0: Ada pengaruh jenis campuran beton terhadap serap air.
